In [41]:
import os
import pandas as pd
from collections import Counter
import re
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [42]:
bg_file = "/home2/s439906/project/CASP16/oligomer/3_stoichiometry/stoich_bg_distribution.csv"


In [ ]:
# 读取 CSV 文件
data = pd.read_csv(bg_file)

# 提取和统计整数信息，并将不同字母前缀的整数视为等价
molecule_counts = Counter()
for _, row in data.iterrows():
    entry = row['stoichiometry']
    count = row['count']
    molecules = entry.split(", ")
    for molecule in molecules:
        # 仅提取整数部分并统计（如 A1 和 C1 视为等价，按 count 列的值累加）
        match = re.search(r'\d+', molecule)
        if match:
            molecule_counts[int(match.group())] += count

# 转换为 DataFrame 并显示结果
molecule_counts_df = pd.DataFrame(molecule_counts.items(), columns=['Integer', 'Count'])
print("整数信息统计（不同字母前缀的整数视为等价，按 count 列的值累加）：")
# print(molecule_counts_df)




In [ ]:
import pandas as pd
from collections import Counter
import re
from itertools import combinations

# 读取 CSV 文件
data = pd.read_csv(bg_file)

# 提取和统计所有pairwise的整数信息
pairwise_counts = Counter()
for _, row in data.iterrows():
    entry = row['stoichiometry']
    count = row['count']
    molecules = entry.split(", ")
    # 提取整数部分
    integers = [re.search(r'\d+', molecule).group() for molecule in molecules if re.search(r'\d+', molecule)]
    if 1 < len(integers) <= 10:
        for pair in combinations(sorted(integers, reverse=True), 2):
            pairwise_counts[pair] += count

# 转换为 DataFrame 并显示结果
pairwise_counts_df = pd.DataFrame(pairwise_counts.items(), columns=['Pair', 'Count'])
print("Pairwise 整数信息统计：")
print(pairwise_counts_df)


In [ ]:
import pandas as pd
from collections import Counter
import re
from itertools import combinations
from math import gcd

# 读取 CSV 文件
data = pd.read_csv(bg_file)

# 提取和统计所有pairwise的整数信息
pairwise_counts = Counter()
for _, row in data.iterrows():
    entry = row['stoichiometry']
    count = row['count']
    molecules = entry.split(", ")
    # 提取整数部分
    integers = [re.search(r'\d+', molecule).group() for molecule in molecules if re.search(r'\d+', molecule)]
    if 1 < len(integers) <= 10:
        for pair in combinations(sorted(integers, reverse=True), 2):
            pairwise_counts[pair] += count

# 转换为 DataFrame
pairwise_counts_df = pd.DataFrame(pairwise_counts.items(), columns=['Pair', 'Count'])

# 处理比例，将等价的组合合并，只在互质情况下保留原值
normalized_pairwise_counts = Counter()
for (a, b), count in pairwise_counts.items():
    a, b = int(a), int(b)
    common_divisor = gcd(a, b)
    if common_divisor > 1:
        normalized_pair = tuple(sorted((a // common_divisor, b // common_divisor), reverse=True))
    else:
        normalized_pair = tuple(sorted((a, b), reverse=True))
    normalized_pairwise_counts[normalized_pair] += count

# 转换为 DataFrame 并显示结果
normalized_pairwise_counts_df = pd.DataFrame(normalized_pairwise_counts.items(), columns=['Pair', 'Count'])
print("Normalized Pairwise 整数信息统计：")
print(normalized_pairwise_counts_df)


In [ ]:
total_count_pair = sum(normalized_pairwise_counts_df['Count'])
normalized_pairwise_counts_df['Count'] /= total_count_pair
normalized_pairwise_counts_df.set_index('Pair', inplace=True)
# convert every element in index to string
normalized_pairwise_counts_df.index = normalized_pairwise_counts_df.index.map(lambda x: tuple(map(str, x)))
normalized_pairwise_counts_df

In [ ]:
total_count_molecule = sum(molecule_counts_df['Count'])
molecule_counts_df['Count'] /= total_count_molecule
# set Integer as index
molecule_counts_df.set_index('Integer', inplace=True)
molecule_counts_df.index = molecule_counts_df.index.map(str)
molecule_counts_df

In [ ]:
stoichiometry_dir = "/data/data1/conglab/jzhan6/CASP16/CASP16_scores/oligo_20240910/firstmodels/"
stoichiometry_files = [file for file in os.listdir(stoichiometry_dir) if file.endswith('.stoichiometry')]
# sort files alphabetically
stoichiometry_files.sort()
stoichiometry_files.__len__()

In [ ]:
for file in stoichiometry_files:
    file = os.path.join(stoichiometry_dir, file)
    data = pd.read_csv(file, sep='\t')
    data['group_info'] = data['model'].str.extract(r'(TS\d{3})')
data

In [ ]:
def extract_single_and_pairwise_combinations(column):
    single_numbers = [int(n) for n in re.findall(r'\d+', column)]
    pairwise_combinations = []
    for a, b in combinations(single_numbers, 2):
        if a == 0 or b == 0:
            pairwise_combinations.append((max(a, b), 0))  
        else:
            common_divisor = gcd(a, b)
            reduced_pair = (a // common_divisor, b // common_divisor)
            pairwise_combinations.append((max(reduced_pair), min(reduced_pair)))
    return single_numbers, pairwise_combinations

def calculate_single_scores(single_list, probability_df):
    return [1 - probability_df.loc[str(n), 'Count'] if str(n) in probability_df.index else 0 for n in single_list]

def calculate_pairwise_scores(pairwise_list, probability_df):
    return [1 - probability_df.loc[tuple(sorted(map(str, pair), reverse=True)), 'Count'] if tuple(sorted(map(str, pair), reverse=True)) in probability_df.index else 0 for pair in pairwise_list]

def calculate_combined_single_score(stoi_single, truth_single, truth_single_scores):
    combined_score = 0
    for i, num in enumerate(truth_single):
        if num in stoi_single:
            combined_score += truth_single_scores[i]
    return combined_score

def calculate_combined_pairwise_score(stoi_pairwise, truth_pairwise, truth_pairwise_scores):
    combined_score = 0
    for i, pair in enumerate(truth_pairwise):
        if pair in stoi_pairwise:
            combined_score += truth_pairwise_scores[i]
    return combined_score


score_all = None
for file in stoichiometry_files:
    file_abs = os.path.join(stoichiometry_dir, file)
    data = pd.read_csv(file_abs, sep='\t')
    data['group_info'] = data['model'].str.extract(r'(TS\d{3})')
    data['stoi_single'] = data['stoi'].apply(lambda x: extract_single_and_pairwise_combinations(x)[0])
    data['stoi_pairwise'] = data['stoi'].apply(lambda x: extract_single_and_pairwise_combinations(x)[1])
    data['truth_single'] = data['truth'].apply(lambda x: extract_single_and_pairwise_combinations(x)[0])
    data['truth_pairwise'] = data['truth'].apply(lambda x: extract_single_and_pairwise_combinations(x)[1])
    data['truth_single_scores'] = data['truth_single'].apply(lambda x: calculate_single_scores(x, molecule_counts_df))
    data['truth_pairwise_scores'] = data['truth_pairwise'].apply(lambda x: calculate_pairwise_scores(x, normalized_pairwise_counts_df))
    data['score_single'] = data.apply(lambda row: calculate_combined_single_score(row['stoi_single'], row['truth_single'], row['truth_single_scores']), axis=1)
    data['score_pairwise'] = data.apply(lambda row: calculate_combined_pairwise_score(row['stoi_pairwise'], row['truth_pairwise'], row['truth_pairwise_scores']), axis=1)
    data['sum_score'] = data['score_single'] + data['score_pairwise']
    # print(data[['model', 'stoi', 'truth', 'stoi_single', 'stoi_pairwise', 'truth_single', 'truth_pairwise', 'truth_single_scores', 'truth_pairwise_scores', 'score_single', 'score_pairwise', 'sum_score']])
    # get group_info and sum_score, and change sum_score name to file.split('.')[-1]
    result = data[['group_info', 'sum_score']]
    result.columns = ['group_info', file.split('.')[0]]
    # set group_info as index
    result.set_index('group_info', inplace=True)
    print(result)
    if score_all is None:
        score_all = result
    else:
        # merge result to score_all
        score_all = pd.concat([score_all, result], axis=1)


In [ ]:
score_all
# sum and sort by sum
score_all['sum'] = score_all.sum(axis=1)
score_all.sort_values(by='sum', ascending=False, inplace=True)
score_all


In [ ]:
normalized_pairwise_counts_df.loc[('2', '1'), 'Count']

In [ ]:
# plot  sum_score as bar plot
import matplotlib.pyplot as plt
plt.figure(figsize=(18, 6))
plt.bar(score_all.index, score_all['sum'])
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=14)
plt.xlabel('Group', fontsize=16)
plt.ylabel('Sum Score', fontsize=16)
plt.title('stoichiometry score', fontsize=20)
plt.show()
